In [29]:
import great_expectations as gx
import pandas as pd
import warnings
from sqlalchemy.exc import SQLAlchemyError
from datetime import datetime, date
warnings.filterwarnings("ignore", message="`result_format` configured at the Validator-level*")

# Load the data
#df = pd.read_csv("./data/transactions.csv")
transaction_df = pd.read_csv("./data/transactions.csv", dtype="string")
account_df = pd.read_csv("./data/sebank_customers_with_accounts.csv", dtype="string")
error_columns = list(transaction_df.columns) + ['error notes']
transaction_error_df = pd.DataFrame(columns=error_columns)

def progress_bar(current, percent):
    temp_count =int(current/percent)
    bar_percent = int(temp_count / 10)
    bar = "█" * bar_percent
    if  percent != 0 and current % percent == 0 :
        if temp_count % 10 >= 5:
            bar = bar + "▌"
        print(f"{temp_count}% {bar}",)

for column in transaction_df:
    if column == "amount": #removes spacing errors from the amount column
        transaction_df[column] = transaction_df[column].astype(str).str.replace(" ","")
    elif column == "timestamp":
        transaction_df[column] = transaction_df[column].astype(str).str.replace("-","")
        transaction_df[column] = transaction_df[column].astype(str).str.replace(".",":")
    transaction_df[column] = transaction_df[column].astype(str).str.strip()
    #print(transaction_df[column].head())

#data conversion
transaction_df["amount"] = pd.to_numeric(transaction_df["amount"], errors="coerce")


for index in range(transaction_df.shape[0]):
    row = transaction_df.loc[index]
    if pd.isna(transaction_df.loc[index, 'notes']):
        transaction_df.loc[index, 'notes'] = "no notes"
    if row.isna().any():
        temp_row = transaction_df.iloc[[index]].copy()
        temp_row = row.fillna("missing data")
        temp_row[index,"error notes"] = "missing data"
        transaction_error_df = pd.concat([transaction_error_df, temp_row], ignore_index=True)
        transaction_df.drop(index, inplace=True)
        print(f"missing data: {index}")
        progress_bar(index, 1000)
        continue
    #end of timestamp empty check
    temp_list = str(transaction_df.loc[index,'timestamp']).split()
    temp_date = temp_list[0]
    temp_time = temp_list[1]
    datetime_corrected = False
    if len(temp_list[0]) == 6:
        datetime_corrected = True
        temp_date = "20" + temp_list[0]
        if datetime.strptime(temp_date, "%Y%m%d").date() > date.today():
            temp_date = "19" + temp_list[0]
            #end of date length check if-statement
    if len(temp_list[1]) == 5:
        datetime_corrected = True
        temp_time = temp_list[1] + ":00"
        #end of date length check if-statement
    if datetime_corrected:
        temp_timestamp = temp_date + " " + temp_time
        transaction_df.loc[index,'timestamp'] = temp_timestamp
    try:
        transaction_df.loc[index,'timestamp'] = pd.to_datetime(transaction_df.loc[index,'timestamp'], format="%Y%m%d %H:%M:%S", errors="raise")
    except ValueError as e:
        temp_row = transaction_df.iloc[[index]].copy()
        temp_row["error notes"] = "invalid timestamp format"
        transaction_error_df = pd.concat([transaction_error_df, temp_row], ignore_index=True)
        transaction_df.drop(index, inplace=True)
        print(f"incorrect timestamp {index}")
        progress_bar(index, 1000)
        continue
    progress_bar(index, 1000)


print(transaction_df.head())
print(transaction_error_df.head())
# Create the ephemeral GX context
context = gx.get_context()

# Add a pandas datasource
data_source = context.data_sources.add_pandas(name="pandas")

# Add a dataframe asset
data_asset = data_source.add_dataframe_asset(name="transactions_data")

# Define the batch (entire DataFrame)
batch_definition = data_asset.add_batch_definition_whole_dataframe(name="batch_def")
batch = batch_definition.get_batch(batch_parameters={"dataframe": transaction_df})

# Create the expectation suite with a name
suite = gx.core.expectation_suite.ExpectationSuite(name="transactions_suite")

# Get the validator using the suite
validator = context.get_validator(batch=batch, expectation_suite=suite)

# Add expectations
validator.expect_column_values_to_be_between("amount", min_value=0.01, max_value=100000)
validator.expect_column_values_to_not_be_null("timestamp")

# Validate
results = validator.validate()

# Print results
print(results)

0% 


C:\Users\cfrif\AppData\Local\Temp\ipykernel_10452\4256989140.py:73: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  transaction_error_df = pd.concat([transaction_error_df, temp_row], ignore_index=True)


incorrect timestamp 821
incorrect timestamp 866
incorrect timestamp 876
incorrect timestamp 884
incorrect timestamp 911
incorrect timestamp 913
1% 
incorrect timestamp 1428
2% 
3% 
4% 


KeyboardInterrupt: 